In [2]:
from google.colab import files
upload=files.upload()
#調達実績（件名、取引先）をアップロード
#tfidf2.pyをアップロード

Saving 201812-202012(utf-8).csv to 201812-202012(utf-8).csv


２つのファイルをアップロードする。
1)自作の関数ライブラリ　tfidf2.py
2)過去の発注データから　件名欄＋仕様欄、発注先、注文番号を抽出したcsvファイルDoc2Vec-test-utf-8.csv

In [3]:
#!apt install aptitude
#!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
#!pip install mecab-python3==0.7
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
import MeCab
import pickle
import numpy as np
import tfidf2

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


形態素解析ツール　MeCab　他　ライブラリをimport

In [4]:
x=[]
y=[]

with open('201812-202012(utf-8).csv', mode='rt',encoding='utf-8') as f:
 for line in f:
   # ,区切りで　件名、業者名、ラベルに分離
    p=line.split(',')
    print("件名=",p[0],"ﾗﾍﾞﾙ=",p[1])
    tfidf2.add_text(p[0])
    y.append(p[1])    
x=tfidf2.calc_files()
pickle.dump([y,x],open('genre.pickle','wb'))
tfidf2.save_dic('genre-tdidf.dic')
print('ok')

ストリーミング出力は最後の 5000 行に切り捨てられました。

件名= SiC前駆体ポリマー【研究開発用材料】品名：StarPCS　SMP-10　　（アリルハイドライドポリカルボシラン：AHPCS）  密度0.998g/cm3  数量：3kg  製造会社：Starfire Systems社 その他：Starfire　Systems社出荷～弊社納入まで温度記録を取得し 提出すること ﾗﾍﾞﾙ= 2009H1C101101-1K75016:ＮＴＫインターナショナル（株）　名古屋支店

件名= 日本道路株式会社殿の岡山合材センターにおけるガス分析詳細は発注仕様書(仕2009H1C1012)に記載。 ﾗﾍﾞﾙ= 2009H1C101201-1K42722:中外テクノス（株）　関西技術センター

件名= ステンレス鋼板の溶接継手部の引張せん断試験依頼書⇒添付資料『仕-2009H1C1013』 数量=4種 ﾗﾍﾞﾙ= 2009H1C101301-1K13697:川重テクノロジー（株）

件名= パウダ収束性評価用ジグ製作仕2009H1C101401  参照 ﾗﾍﾞﾙ= 2009H1C101401-1K37390:（有）田中工作所

件名= アルカリ脆化SCC予亀裂導入試験詳細は仕様書（仕_2009H1C101501）をご参照願います。 ﾗﾍﾞﾙ= 2009H1C101501-1K13697:川重テクノロジー（株）

件名= ステンレス鋼のフェライトスコープ測定添付資料「仕2009H1C101601_ステンレス鋼のフェライトスコープ測定」を参照のこと。 ﾗﾍﾞﾙ= 2009H1C101601-1K13697:川重テクノロジー（株）

件名= ICFB層内伝熱管-溶接部に関する高温腐食摩耗試験⑨添付ファイルを参照 ﾗﾍﾞﾙ= 2009H1C101701-1K13697:川重テクノロジー（株）

件名= エタノール品名：エタノール メーカー：富士フイルム和光純薬 容量：18L 等級：1級 販売元コード：052-00467 数量：2缶 ﾗﾍﾞﾙ= 2009H1C101801-1K65250:広瀬化学薬品（株）　兵庫西支店

件名= 自動研磨機用ダイヤモンドディスク自動研磨機にて使用する研磨紙を購入する。  １．メーカ：プレシ社 　　品名：ダイヤモンドディスク I-MAX RΦ300m

過去データをreadして　変数へ格納
x：件名＋仕様を形態素解析して　各単語の頻出度*重要度を値とする多次元配列（ベクトル）へ変換
　y:発注先＋注文番号
　ファイルへデータを書き込む

In [5]:
n=pickle.load(open('genre-tdidf.dic','rb'))
word_dic,dt_dic,files=n
#print(word_dic)
#print(dt_dic)
#print(files)

データの読み込み

In [7]:
#train_db
import pickle
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import numpy as np
# TF-IDFデータベースを読み込む
data=pickle.load(open("genre.pickle","rb"))
y=data[0]
x=data[1]


データを　x　、ｙへ格納

In [23]:
#ベクトル化文章のアレイ化
xx=np.array(x)

計算のため　変数ｘをアレイ化

In [24]:
print(xx)

[[0.25047297 0.5502041  0.34936802 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.93975146]]


In [10]:
#関数定義
#検索文章のベクトル化
def check_genre(text):
  data=tfidf2.calc_text(text)
  return data
#コサイン類似度計算
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

検索文字を形態素解析して　頻出度＊重要度で　多次元配列化（ベクトル化）する関数
過去データのベクトルと検索文字のベクトルでコサイン類似度を計算する関数

In [20]:
#検索文章の設定
text1="動画編集用SSD  "
dt=check_genre(text1)
print(dt)

[0. 0. 0. ... 0. 0. 0.]


In [21]:
#類似案件　検索
#初期値設定
max=[0,0,0]
imax=[0,0,0]

#文章ベクトルアレイ化
Y = np.array(dt)
for i in range(len(xx)):
  X = np.array(xx[i])
  tmp= cos_sim(X, Y)
#  print(tmp) 
  if tmp>max[0] :
    max[2]=max[1]
    max[1]=max[0]
    max[0]=tmp
    imax[2]=imax[1]
    imax[1]=imax[0]
    imax[0]=i
  elif max[0] >tmp>max[1]:
    max[2]=max[1]
    max[1]=tmp
    imax[2]=imax[1]
    imax[1]=i
  elif max[1] > tmp > max[2]:
    max[2]=tmp
    imax[2]=i
    
#ベスト３を表示
print(max[0],imax[0],y[imax[0]])
print(max[1],imax[1],y[imax[1]])
print(max[2],imax[2],y[imax[2]])



0.3935970424110301 10101 2001F78000204-1M10059:岡本無線電機（株）西宮（営）

0.3190557234894929 8325 1910HX4028801-1K16239:協立電機（株）

0.3069753734931755 12832 2004F78008701-1M10059:岡本無線電機（株）西宮（営）



コサイン類似度が大きい案件ベスト３を抽出する。

In [ ]:
import csv
with open('word_dic.csv', mode='w',encoding='utf-8') as g:
  writer = csv.writer(g)
  writer.writerows(word_dic)